# Fiscal Data Preprocessing

In [1]:
# Automatic File Naming Utility
from output_util import OutputUtil

outputUtil = OutputUtil()

In [2]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


class FeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, features: list[str]):
        self.features = features

    def fit(self, x: pd.DataFrame, y=None):
        return self

    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        return x.drop(self.features, axis=1)


class MissingValuesImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns: list[str]):
        self.columns = columns

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        imputer = SimpleImputer(strategy='constant', fill_value=0)

        for column in self.columns:
            x[column] = imputer.fit_transform(x[[column]])

        return x


class OperatingSurplusImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        x["Operating surplus (deficit) before financing and transfers"] = x["Total Revenues"] - x["Total Expenses"]
        return x


class YearTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        x["Year"] = x["Year"].str.extract(r"(\d{4})-.*")
        return x


class ColumnsShrink(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        x["Conditional transfers from other governments"] += x[
                                                                 "Conditional Transfers From Federal Or Provincial Governments Or Agencies"] + \
                                                             x["Conditional Transfers From Other Local Governments"]

        x["Net property taxes and payments in lieu of taxes"] += x[
            "Assessable Property Taxes (Net Of School Board Appropriations, corrections, housing deficits)"]

        return x

# Pipeline

In [3]:
from globals import FISCAL_PATH

tax_columns = [
    "Net property taxes and payments in lieu of taxes",
    "Assessable Property Taxes (Net Of School Board Appropriations, corrections, housing deficits)"
]

transfers_columns = [
    "Conditional transfers from other governments",
    "Conditional Transfers From Federal Or Provincial Governments Or Agencies",
    "Conditional Transfers From Other Local Governments"
]

features_to_drop = [
    "TOTAL FINANCING AND TRANSFERS",
    "ANNUAL SURPLUS (DEFICIT)",
    "Conditional Transfers From Federal Or Provincial Governments Or Agencies",
    "Conditional Transfers From Other Local Governments",
    "Assessable Property Taxes (Net Of School Board Appropriations, corrections, housing deficits)"
]

# Pipeline
pipe = Pipeline([
    ('tax imputer', MissingValuesImputer(columns=tax_columns)),
    ('transfer imputer', MissingValuesImputer(columns=transfers_columns)),
    ('operating surplus imputer', OperatingSurplusImputer()),
    ('transfers shrink', ColumnsShrink()),
    ('year transformer', YearTransformer()),
    ('drop', FeatureDropper(features=features_to_drop)),

])
output_df = pipe.fit_transform(pd.read_csv(FISCAL_PATH))

output_df.to_csv(outputUtil.generate_output_filepath(desc="fiscal"), index=False)

# Outliers

In [4]:
fiscal_df = pd.read_csv(outputUtil.get_curr_filepath())

grouped_count = fiscal_df.groupby(['Region', 'Region Type']).size().reset_index(name='Count')
# grouped_count = grouped_count[grouped_count['Count'] != 8]
grouped_count

,Region,Region Type,Count
0,Amherst,Town,8
1,Annapolis,Rural Municipality,7
2,Annapolis,Town,1
3,Annapolis Royal,Town,8
4,Antigonish,Rural Municipality,8
5,Antigonish,Town,8
6,Argyle,Rural Municipality,8
7,Barrington,Rural Municipality,8
8,Berwick,Town,8
9,Bridgetown,Town,2


# Findings
Issues have been identified with following cities:

| City       | Issue                                               | Proposed Fix                                                                          |
|------------|-----------------------------------------------------|---------------------------------------------------------------------------------------|
| Annapolis  | Changed from Rural Municipality to Town in 2020     | Combine all the Annapolis (Annapolis + Annapolis data) and rename to Annapolis County |
| Bridgetown | All data is not available                           | Merge with Annapolis County data                                                      |
| Hantsport  | All data is not available                           | Merge with West Hants data                                                            |
| Parrsboro  | All data is not available                           | Merge with Cumberland data                                                            |
| Springhill | All data is not available                           | Merge with Cumberland data                                                            |
| West Hants | Changes Rural Municipality to Regional Municipality | Merge with other data                                                                 |
| Windsor    | All data is not available                           | Merge with West Hants data                                                            |


| City       | Relevant Notes                                                                                                                   |
|------------|----------------------------------------------------------------------------------------------------------------------------------|
| Annapolis  | Annapolis and Annapolis Royal are part of larger County of Annapolis                                                             |
| Bridgetown | In 2015, it dissolved into the larger Municipality of the County of Annapolis.                                                   |
| Hantsport  | In 2014, it dissolved into the larger Municipality of West Hants                                                                 |
| Parrsboro  | In 2016, it dissolved into the larger Municipality of Cumberland                                                                 |
| Springhill | In 2015 the Town of Springhill amalgamated into the Municipality of the County of Cumberland.                                    |
| West Hants | On April 1, 2020, the Town of Windsor amalgamated with the District of West Hants to become the West Hants Regional Municipality |
| Windsor    | On April 1, 2020, the Town of Windsor amalgamated with the District of West Hants to become the West Hants Regional Municipality |


# Merging into Counties

In [5]:
fiscal_df.groupby(["Region"]).size().reset_index(name='Count')

,Region,Count
0,Amherst,8
1,Annapolis,8
2,Annapolis Royal,8
3,Antigonish,16
4,Argyle,8
5,Barrington,8
6,Berwick,8
7,Bridgetown,2
8,Bridgewater,8
9,CBRM,8


Entities to be merged because they fall under the same county:

| Town        | Rural Municipality | County     |
|-------------|--------------------|------------|
| Annapolis   | Annapolis Royal    | Annapolis  |
| Antigonish  | Antigonish         | Antigonish |
| Digby       | Digby              | Digby      |
| Lunenberg   | Lunenberg          | Lunenberg  |
| Pictou      | Pictou             | Pictou     |
| Shelburne   | Shelburne          | Shelburne  |
| Yarmouth    | Yarmouth           | Yarmouth   |

# Processing

In [6]:
fiscal_df = pd.read_csv(outputUtil.get_curr_filepath())

regions_to_counties = {
    'Annapolis': ['Annapolis', 'Annapolis Royal', 'Bridgetown', 'Middleton'],
    'Colchester': ['Truro', 'Stewiacke'],
    'Cumberland': ['Springhill', 'Parrsboro', 'Amherst', 'Oxford'],
    'Digby': ['Clare'],
    'Guysborough': ["St. Mary's", 'Mulgrave'],
    'Hants': ['Hantsport', 'Windsor', 'West Hants', 'East Hants'],
    'Halifax': ['HRM'],
    'Kings': ['Berwick', 'Kentville', 'Wolfville'],
    'Lunenburg': ['Chester', 'Bridgewater', 'Mahone Bay'],
    'Pictou': ['New Glasgow', 'Stellarton', 'Trenton', 'Westville'],
    'Queens': ['Region of Queens'],
    'Shelburne': ["Clark's Harbour", 'Lockeport', 'Barrington'],
    'Yarmouth': ['Argyle'],
    'Cape Breton': ['Port Hawkesbury', 'CBRM']
}

for county in regions_to_counties.keys():
    fiscal_df.loc[fiscal_df["Region"].isin(regions_to_counties[county]), "Region"] = county

fiscal_df = fiscal_df.groupby(['Year', 'Region'], as_index=False).sum()

region_type_to_counties = {
    'County': ['Annapolis', 'Antigonish', 'Colchester', 'Cumberland', 'Inverness', 'Kings', 'Pictou', 'Richmond',
               'Victoria'],
    'Regional': ['Cape Breton', 'Halifax', 'Queens'],
    'District': ['Digby', 'Guysborough', 'Hants', 'Lunenburg', 'Shelburne', 'Yarmouth']
}

for region_type in region_type_to_counties.keys():
    fiscal_df.loc[fiscal_df['Region'].isin(region_type_to_counties[region_type]), 'Region Type'] = region_type

fiscal_df = fiscal_df.rename(columns={'Region Type': 'Municipality Type'})

# Next year fields
fiscal_df.sort_values(by="Year", ascending=True, inplace=True)
fiscal_df["Next Year Total Expenses"] = fiscal_df.groupby('Region')['Total Expenses'].shift(-1)
fiscal_df["Next Year Total Revenues"] = fiscal_df.groupby('Region')['Total Revenues'].shift(-1)

fiscal_df.to_csv(outputUtil.generate_output_filepath(desc="fiscal_regions_processed"), index=False)

# Municipality Type Encoding

In [7]:
class MunicipalityTypeEncoding(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, x: pd.DataFrame, y=None):
        return self

    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        one_hot = pd.get_dummies(x['Municipality Type'])
        x = x.drop(['Municipality Type'], axis=1)
        x = x.join(one_hot)
        return x


fiscal_df = pd.read_csv(outputUtil.get_curr_filepath())

# Pipeline
pipe = Pipeline([
    ('onehot', MunicipalityTypeEncoding()),
])

output_df = pipe.fit_transform(fiscal_df)
output_df.to_csv(outputUtil.generate_output_filepath(desc="one_hot"), index=False)

# Final Output of the Notebook

In [8]:
df = pd.read_csv(outputUtil.get_curr_filepath())
df.to_csv(outputUtil.get_final_filepath(), index=False)